<a href="https://colab.research.google.com/github/TatKhachatryan/Lionel-Messi-Statistics-dash-app/blob/main/Messi_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q dash
!pip install -q jupyter_dash
!pip install -q dash-cytoscape

In [2]:
import pandas as pd
import numpy as np

from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, callback, Output, Input

import plotly.express as px
import plotly.graph_objects as go

In [3]:
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [4]:
#defining my worksheet
sh = gc.open('messi_starplayerstats')
sh2 = gc.open('Messi_Fbref_live_data')
# sh2.worksheets()
sh.worksheets()

[<Worksheet 'all_time_games' id:0>,
 <Worksheet 'assists_to' id:970228274>,
 <Worksheet 'assists_from' id:709529979>,
 <Worksheet 'opponents' id:1805269916>,
 <Worksheet 'all_time_goals' id:1480681430>,
 <Worksheet 'first_goal_of_the_game' id:84730686>,
 <Worksheet 'first_goal_of_the_team' id:1989863561>,
 <Worksheet 'last_goal_of_the_game' id:18567117>,
 <Worksheet 'game_decider' id:316905952>,
 <Worksheet 'last_goal_of_the_team' id:1719246680>,
 <Worksheet 'stoppage_time_goals' id:1414313092>,
 <Worksheet 'Camp_Nou_goals' id:1255631610>,
 <Worksheet 'wins' id:965109892>,
 <Worksheet 'draws' id:1869875317>,
 <Worksheet 'loses' id:238180176>]

In [5]:
# domestic_league = pd.DataFrame(sh2.worksheet("domestic_league").get_all_records())
# domestic_cups = pd.DataFrame(sh2.worksheet("domestic_cups").get_all_records())
# international_league = pd.DataFrame(sh2.worksheet("international_league").get_all_records())
# national_team = pd.DataFrame(sh2.worksheet("national_team").get_all_records())

In [6]:
all_time_games = pd.DataFrame(sh.worksheet("all_time_games").get_all_records())
all_time_goals = pd.DataFrame(sh.worksheet("all_time_goals").get_all_records())
first_goal_of_the_game = pd.DataFrame(sh.worksheet("first_goal_of_the_game").get_all_records())
last_goal_of_the_game = pd.DataFrame(sh.worksheet("last_goal_of_the_game").get_all_records())
first_goal_of_the_team = pd.DataFrame(sh.worksheet("first_goal_of_the_team").get_all_records())
last_goal_of_the_team = pd.DataFrame(sh.worksheet("last_goal_of_the_team").get_all_records())
game_decider = pd.DataFrame(sh.worksheet("game_decider").get_all_records())
stoppage_time_goals = pd.DataFrame(sh.worksheet("stoppage_time_goals").get_all_records())
Camp_Nou_goals = pd.DataFrame(sh.worksheet("Camp_Nou_goals").get_all_records())
wins = pd.DataFrame(sh.worksheet("wins").get_all_records())
draws = pd.DataFrame(sh.worksheet("draws").get_all_records())
loses = pd.DataFrame(sh.worksheet("loses").get_all_records())
assists_to = pd.DataFrame(sh.worksheet("assists_to").get_all_records())
assists_from = pd.DataFrame(sh.worksheet("assists_from").get_all_records())
opponents = pd.DataFrame(sh.worksheet("opponents").get_all_records())

In [7]:
all_time_games.columns = ['#', 'Date', 'Competition', 'Home team', 'Result', 'Away team', 'Lineup', 'Minutes', 'Goals', 'Assists', 'Cards', 'Jersey']
all_time_goals.columns = ['#', 'Date', 'Competition', 'Home team',  'Result', 'Away team', 'Minute', 'Score', 'What', 'How', 'Jersey']
first_goal_of_the_game.columns = ['#', 'Date', 'Competition', 'Home team',  'Result', 'Away team', 'Minute', 'Score', 'What', 'How', 'Jersey']
last_goal_of_the_game.columns = ['#', 'Date', 'Competition', 'Home team',  'Result', 'Away team', 'Minute', 'Score', 'What', 'How', 'Jersey']
first_goal_of_the_team.columns = ['#', 'Date', 'Competition', 'Home team',  'Result', 'Away team', 'Minute', 'Score', 'What', 'How', 'Jersey']
last_goal_of_the_team.columns = ['#', 'Date', 'Competition', 'Home team',  'Result', 'Away team', 'Minute', 'Score', 'What', 'How', 'Jersey']
game_decider.columns = ['#', 'Date', 'Competition', 'Home team',  'Result', 'Away team', 'Minute', 'Score', 'What', 'How', 'Jersey']
stoppage_time_goals.columns = ['#', 'Date', 'Competition', 'Home team',  'Result', 'Away team', 'Minute', 'Score', 'What', 'How', 'Jersey']
Camp_Nou_goals.columns = ['#', 'Date', 'Competition', 'Home team', 'Result', 'Away team', 'Lineup', 'Minutes', 'Goals', 'Assists', 'Cards']
wins.columns = ['#', 'Date', 'Competition', 'Home team', 'Result', 'Away team', 'Lineup', 'Minutes', 'Goals', 'Assists', 'Cards', 'Jersey']
draws.columns = ['#', 'Date', 'Competition', 'Home team', 'Result', 'Away team', 'Lineup', 'Minutes', 'Goals', 'Assists', 'Cards', 'Jersey']
loses.columns = ['#', 'Date', 'Competition', 'Home team', 'Result', 'Away team', 'Lineup', 'Minutes', 'Goals', 'Assists', 'Cards', 'Jersey']
assists_to.columns = ['#', 'Date', 'Competition', 'Home', 'Result', 'Away', 'Minute', 'Assist on', 'Score']
assists_from.columns = ['#', 'Date', 'Competition', 'Home', 'Result', 'Away', 'Minute', 'Assist on', 'Score']
opponents.columns = ['#', 'Opponents', 'Country', 'Games', 'Wins', 'Draws', 'Defeats', 'Winning percentage',
                     'Goals', 'Coefficient', 'Assists', 'Cards', 'Penalties', 'Penalties missed']

In [8]:
# convert Date to datetime
all_time_games['Date'] = pd.to_datetime(all_time_games['Date'])
# set team names
all_time_games['Team'] = np.where((all_time_games['Home team']=='Paris Saint-Germain') | (all_time_games['Away team']=='Paris Saint-Germain'), "Paris Saint-Germain",
                                  np.where(((all_time_games['Home team']=='FC Barcelona') | (all_time_games['Away team']=='FC Barcelona')),"FC Barcelona", "Argentina"))

# set seasons
all_time_games['Year'] = all_time_games['Date'].dt.year
all_time_games['Season'] = np.where(all_time_games['Year']==2023, '2022-23',
                                    np.where(all_time_games['Year']==2022, '2021-22',
                                             np.where(all_time_games['Year']==2021, '2020-21',
                                                      np.where(all_time_games['Year']==2020, '2019-20',
                                                               np.where(all_time_games['Year']==2019, '2018-19',
                                                                        np.where(all_time_games['Year']==2018, '2017-18',
                                                                                 np.where(all_time_games['Year']==2017, '2016-17',
                                                                                          np.where(all_time_games['Year']==2016, '2015-16',
                                                                                                   np.where(all_time_games['Year']==2015, '2014-2015',
                                                                                                            np.where(all_time_games['Year']==2014, '2013-14',
                                                                                                                     np.where(all_time_games['Year']==2013, '2012-13',
                                                                                                                              np.where(all_time_games['Year']==2012, '2011-12',
                                                                                                                                       np.where(all_time_games['Year']==2011, '2010-11',
                                                                                                                                                np.where(all_time_games['Year']==2010, '2009-10',
                                                                                                                                                         np.where(all_time_games['Year']==2009, '2008-09',
                                                                                                                                                                  np.where(all_time_games['Year']==2008, '2007-08',
                                                                                                                                                                           np.where(all_time_games['Year']==2007, '2006-07',
                                                                                                                                                                                    np.where(all_time_games['Year']==2006, '2005-06', '2004-05'))))))))))))))))))



all_time_goals['Date'] = pd.to_datetime(all_time_goals['Date'])
all_time_goals['What']= all_time_goals['What'].replace({'': 'Blank'})
# set team names
all_time_goals['Team'] = np.where((all_time_goals['Home team']=='Paris Saint-Germain') | (all_time_goals['Away team']=='Paris Saint-Germain'), "Paris Saint-Germain",
                                  np.where(((all_time_goals['Home team']=='FC Barcelona') | (all_time_goals['Away team']=='FC Barcelona')),"FC Barcelona", "Argentina"))

# set seasons
all_time_goals['Year'] = all_time_goals['Date'].dt.year
all_time_goals['Season'] = np.where(all_time_goals['Year']==2023, '2022-23',
                                    np.where(all_time_goals['Year']==2022, '2021-22',
                                             np.where(all_time_goals['Year']==2021, '2020-21',
                                                      np.where(all_time_goals['Year']==2020, '2019-20',
                                                               np.where(all_time_goals['Year']==2019, '2018-19',
                                                                        np.where(all_time_goals['Year']==2018, '2017-18',
                                                                                 np.where(all_time_goals['Year']==2017, '2016-17',
                                                                                          np.where(all_time_goals['Year']==2016, '2015-16',
                                                                                                   np.where(all_time_goals['Year']==2015, '2014-2015',
                                                                                                            np.where(all_time_goals['Year']==2014, '2013-14',
                                                                                                                     np.where(all_time_goals['Year']==2013, '2012-13',
                                                                                                                              np.where(all_time_goals['Year']==2012, '2011-12',
                                                                                                                                       np.where(all_time_goals['Year']==2011, '2010-11',
                                                                                                                                                np.where(all_time_goals['Year']==2010, '2009-10',
                                                                                                                                                         np.where(all_time_goals['Year']==2009, '2008-09',
                                                                                                                                                                  np.where(all_time_goals['Year']==2008, '2007-08',
                                                                                                                                                                           np.where(all_time_goals['Year']==2007, '2006-07',
                                                                                                                                                                                    np.where(all_time_goals['Year']==2006, '2005-06', '2004-05'))))))))))))))))))



# convert Date to datetime
assists_to['Date'] = pd.to_datetime(assists_to['Date'])
# set team names
assists_to['Team'] = np.where((assists_to['Home']=='Paris Saint-Germain') | (assists_to['Away']=='Paris Saint-Germain'), "Paris Saint-Germain",
                                  np.where(((assists_to['Home']=='FC Barcelona') | (assists_to['Away']=='FC Barcelona')),"FC Barcelona", "Argentina"))

# set seasons
assists_to['Year'] = assists_to['Date'].dt.year
assists_to['Season'] = np.where(assists_to['Year']==2023, '2022-23',
                                    np.where(assists_to['Year']==2022, '2021-22',
                                             np.where(assists_to['Year']==2021, '2020-21',
                                                      np.where(assists_to['Year']==2020, '2019-20',
                                                               np.where(assists_to['Year']==2019, '2018-19',
                                                                        np.where(assists_to['Year']==2018, '2017-18',
                                                                                 np.where(assists_to['Year']==2017, '2016-17',
                                                                                          np.where(assists_to['Year']==2016, '2015-16',
                                                                                                   np.where(assists_to['Year']==2015, '2014-2015',
                                                                                                            np.where(assists_to['Year']==2014, '2013-14',
                                                                                                                     np.where(assists_to['Year']==2013, '2012-13',
                                                                                                                              np.where(assists_to['Year']==2012, '2011-12',
                                                                                                                                       np.where(assists_to['Year']==2011, '2010-11',
                                                                                                                                                np.where(assists_to['Year']==2010, '2009-10',
                                                                                                                                                         np.where(assists_to['Year']==2009, '2008-09',
                                                                                                                                                                  np.where(assists_to['Year']==2008, '2007-08',
                                                                                                                                                                           np.where(assists_to['Year']==2007, '2006-07',
                                                                                                                                                                                    np.where(assists_to['Year']==2006, '2005-06', '2004-05'))))))))))))))))))


# convert Date to datetime
assists_from['Date'] = pd.to_datetime(assists_from['Date'])
# set team names
assists_from['Team'] = np.where((assists_from['Home']=='Paris Saint-Germain') | (assists_from['Away']=='Paris Saint-Germain'), "Paris Saint-Germain",
                                  np.where(((assists_from['Home']=='FC Barcelona') | (assists_from['Away']=='FC Barcelona')),"FC Barcelona", "Argentina"))

# set seasons
assists_from['Year'] = assists_from['Date'].dt.year
assists_from['Season'] = np.where(assists_from['Year']==2023, '2022-23',
                                    np.where(assists_from['Year']==2022, '2021-22',
                                             np.where(assists_from['Year']==2021, '2020-21',
                                                      np.where(assists_from['Year']==2020, '2019-20',
                                                               np.where(assists_from['Year']==2019, '2018-19',
                                                                        np.where(assists_from['Year']==2018, '2017-18',
                                                                                 np.where(assists_from['Year']==2017, '2016-17',
                                                                                          np.where(assists_from['Year']==2016, '2015-16',
                                                                                                   np.where(assists_from['Year']==2015, '2014-2015',
                                                                                                            np.where(assists_from['Year']==2014, '2013-14',
                                                                                                                     np.where(assists_from['Year']==2013, '2012-13',
                                                                                                                              np.where(assists_from['Year']==2012, '2011-12',
                                                                                                                                       np.where(assists_from['Year']==2011, '2010-11',
                                                                                                                                                np.where(assists_from['Year']==2010, '2009-10',
                                                                                                                                                         np.where(assists_from['Year']==2009, '2008-09',
                                                                                                                                                                  np.where(assists_from['Year']==2008, '2007-08',
                                                                                                                                                                           np.where(assists_from['Year']==2007, '2006-07',
                                                                                                                                                                                    np.where(assists_from['Year']==2006, '2005-06', '2004-05'))))))))))))))))))
# sort by most scored games
opponents = opponents.sort_values(by='Goals', ascending=False, ignore_index=True)

<ipython-input-8-9f831f890292>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  all_time_games['Date'] = pd.to_datetime(all_time_games['Date'])
<ipython-input-8-9f831f890292>:30: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  all_time_goals['Date'] = pd.to_datetime(all_time_goals['Date'])
<ipython-input-8-9f831f890292>:60: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  assists_to['Date'] = pd.to_datetime(assists_to['Date'])
<ipython-input-8-9f831f890292>:88: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This

# Cards

In [9]:
all_time_goals['How'].value_counts()
total_games = all_time_games.shape[0]
total_wins = wins.shape[0]
total_draws = draws.shape[0]
total_lost = loses.shape[0]
red_cards = 3
yellow_cards = all_time_games['Cards'].sum()-red_cards

print('Total games:', total_games)
print('Total wins:', total_wins)
print('Total draws:', total_draws)
print('Total lost:', total_lost)
print('Total red cards:', red_cards)
print('Total yellow cards:', yellow_cards)

Total games: 1021
Total wins: 695
Total draws: 194
Total lost: 132
Total red cards: 3
Total yellow cards: 92


In [10]:
# left or rigth side of the cards
teams = all_time_games.groupby(['Team'])['Team'].count().rename('Count').to_frame()
teams = teams.reset_index()
teams = teams.sort_values(by='Count', ascending=False)

fig = px.bar(teams, x='Count', y='Team', color='Team', orientation='h',
             color_discrete_sequence=['#DB0030', '#43A1D5', '#004170'],
             text_auto='.2s')
fig.show()

# Goals Stats

In [11]:
goals = all_time_games.groupby(['Year'])['Goals'].sum().to_frame()
goals = goals.reset_index()
assists = all_time_games.groupby(['Year'])['Assists'].sum().to_frame()
assists = assists.reset_index()

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=goals['Year'], y=goals['Goals'],
                    mode='lines',
                    name='Goals',
                    line=dict(color='#FECA29')))

fig.add_trace(go.Scatter(x=assists['Year'], y=assists['Assists'],
                    mode='lines',
                    name='Assists',
                    line=dict(color='#757575')))
fig.show()

In [12]:
# input season
goals_by_minutes = all_time_goals[all_time_goals['Season']=='2011-12'].groupby('Minute')['Minute'].count().rename('Count').to_frame()
goals_by_minutes = goals_by_minutes.reset_index()
fig = px.line(goals_by_minutes, x="Minute", y="Count", title='Goals by Minutes for season 2011-12')
fig.show()

In [13]:
# left foot / right foot etc.
goals_how = all_time_goals.groupby(['Season', 'How'])['How'].count().rename('Count').to_frame()
goals_how = goals_how.reset_index()
goals_how = goals_how.sort_values(by='Count', ascending=True)
fig = px.bar(goals_how[goals_how['Season']=='2013-14'],
             x='Count', y='How', text='How', orientation='h',
             width=750)
fig.update_layout(yaxis={'showticklabels': False})
fig.show()

In [14]:
# goal types
goals_what = all_time_goals.groupby(['Season', 'What'])['What'].count().rename('Count').to_frame()
goals_what = goals_what.reset_index()
goals_what = goals_what.sort_values(by='Count', ascending=True)

fig = px.bar(goals_what[goals_what['Season']=='2013-14'],
             x='Count', y='What', text='What', orientation='h',
             width=750)
fig.update_layout(yaxis={'showticklabels': False})
fig.show()

In [15]:
# set input value for seasons
assists_to_df = assists_to.groupby(['Assist on'])['Assist on'].count().rename('Count').to_frame()
assists_to_df = assists_to_df.reset_index()
assists_to_df.columns = ['player', 'to count']
assists_from_df = assists_from.groupby(['Assist on'])['Assist on'].count().rename('Count').to_frame()
assists_from_df = assists_from_df.reset_index()
assists_from_df.columns = ['player', 'from count']

total_assists = pd.merge(assists_to_df, assists_from_df, on='player', how='inner')
total_assists = total_assists.sort_values(['from count', 'to count'], ascending=False, ignore_index=True)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=total_assists['player'][:11],
    y=total_assists['to count'][:11],
    name='Assists to'))

fig.add_trace(go.Bar(
    x=total_assists['player'][:11],
    y=total_assists['from count'][:11],
    name='Assists from',
))

fig.update_layout(barmode='stack', xaxis_tickangle=-45)
fig.show()

# Games Stats

In [16]:
competitions = all_time_games.groupby(['Competition', 'Team'])['Competition'].count().rename('Count').to_frame()
competitions = competitions.reset_index()
competitions['Competition'] = competitions['Competition'].replace({'CPA Copa América 2007': 'Copa América',
                                                            'CPA Copa América 2011': 'Copa América',
                                                            'CPA Copa América 2015': 'Copa América',
                                                            'CPA Copa América 2019': 'Copa América',
                                                            'CPA Copa América 2021': 'Copa América',
                                                            'CPA Copa América Centenario': 'Copa América',
                                                            'WCQ World Cup 2006 Qualification': 'World Cup Qualification',
                                                            'WCQ World Cup 2010 Qualification': 'World Cup Qualification',
                                                            'WCQ World Cup 2014 Qualification': 'World Cup Qualification',
                                                            'WCQ World Cup 2018 Qualification': 'World Cup Qualification',
                                                            'WCQ World Cup 2022 Qualification': 'World Cup Qualification',
                                                            'WCP World Cup 2006': 'World Cup',
                                                            'WCP World Cup 2010': 'World Cup',
                                                            'WCP World Cup 2014': 'World Cup',
                                                            'WCP World Cup 2018': 'World Cup',
                                                            'WCP World Cup 2022': 'World Cup',
                                                            'CUP Copa del Rey': 'Copa del Rey',
                                                            'CUP Coupe de France': 'Coupe de France',
                                                            'EUR UEFA Champions League': 'UEFA Champions League',
                                                            'FRN International friendly': 'International friendly',
                                                            'IUP UEFA European Super Cup': 'UEFA European Super Cup',
                                                            'PRM La Liga': 'La Liga',
                                                            'PRM Ligue 1': 'Ligue 1',
                                                            'SUP Spanish Super Cup': 'Spanish Super Cup',
                                                            'SUP Trophée des Champions': 'Trophée des Champions',
                                                            'WCT FIFA Club World Cup': 'FIFA Club World Cup',
                                                            'COC CONMEBOL–UEFA Cup of Champions 2022': 'UEFA Cup of Champions'
                                                            })

competitions = competitions.sort_values(by='Count', ascending=False)

competitions = competitions.groupby(['Competition', 'Team'])['Count'].sum().to_frame()
competitions = competitions.sort_values(by='Count', ascending=False)
competitions.reset_index(inplace=True)

In [17]:
fig = px.bar(competitions, x='Competition', y='Count', color='Team',
             color_discrete_sequence=['#DB0030', '#43A1D5', '#004170'],
             text_auto='.2s')
fig.show()

In [18]:
year_teams = all_time_games.groupby(['Competition', 'Team', 'Date'])['Competition'].count().rename('Count').to_frame()
year_teams = year_teams.reset_index()
year_teams['Date'] = year_teams['Date'].dt.year

year_teams = year_teams.groupby(['Date', 'Team'])['Count'].sum().to_frame()
year_teams = year_teams.sort_values(by='Count', ascending=False)
year_teams.reset_index(inplace=True)

fig = px.bar(year_teams, x='Date', y='Count', color='Team',
             color_discrete_sequence=['#DB0030', '#43A1D5', '#004170'],
             text_auto='.2s',
             pattern_shape="Team", pattern_shape_sequence=["\\", "|", ""])
fig.show()

In [19]:
opponents['Winning %'] = (opponents['Wins']/opponents['Games'])*100

In [20]:
# opponents
fig = px.bar(opponents[:8], x='Opponents', y='Winning %', width=750, title='Top 5: Goals by Opponent')
fig.show()

In [21]:
# opponents
fig = px.bar(opponents[:5], x='Opponents', y='Goals', width=750,text_auto='.2s', title='Top 5: Goals by Opponent')
fig.show()

# Dash App

In [34]:
goals_what

,Season,What,Count
0,2004-05,Blank,1
77,2019-20,Free kick,1
71,2017-18,Solo run,1
70,2017-18,Rebound,1
61,2015-16,Solo run,1
...,...,...,...
67,2017-18,Field goal,37
27,2010-11,Blank,39
57,2015-16,Field goal,41
21,2009-10,Blank,44


In [36]:
# add html components and figure to app

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1(children='Lionel Messi General Statistics', style={'textAlign':'center'}),
    dcc.Dropdown(goals_what['Season'].unique(), '2011-12', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = goals_what[goals_what['Season']==value]
    fig = px.bar(dff, x='Count', y='What', text='What', orientation='h', width=750)
    fig.update_layout(yaxis={'showticklabels': False})  
    return fig

if __name__ == '__main__':
    app.run_server(mode='external') #"inline" will output the dashboard in colab, "external" will output it in your browser via localhost

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>